In [1]:
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
import networkx as nx

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar,StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
nx_graph = nx.read_graphml("2012-12-02.graphml")

In [3]:
nx.is_weighted(nx_graph)

False

In [88]:
nx.set_edge_attributes(nx_graph, nx.get_edge_attributes(nx_graph, "n_tx"), name="weight")

In [89]:
pr = nx.pagerank(nx_graph, alpha=0.9)
my_df = pd.DataFrame(pr,index=[0]).T

In [90]:
nx.set_node_attributes(nx_graph, pr,'pr')

In [139]:
dg = dict(nx.degree(nx_graph))
data = pd.DataFrame(dg,index=[0]).T
data.columns = ['degree']
data

,degree
81567039,2
81567155,3
81567156,1
81526809,2
79800777,1138
...,...
81573964,3
81573965,3
81573966,4
81573967,3


In [140]:
ntx = dict(nx.degree(nx_graph,weight="n_tx"))
data2 = pd.DataFrame(ntx,index=[0]).T
data2.columns = ['ntx']
data2

,ntx
81567039,2
81567155,3
81567156,1
81526809,2
79800777,1654
...,...
81573964,3
81573965,3
81573966,4
81573967,3


In [141]:
ntx = dict(nx.degree(nx_graph,weight="value"))
data3 = pd.DataFrame(ntx,index=[0]).T
data3.columns = ['value']
data3

,value
81567039,1.139281e+12
81567155,2.278358e+12
81567156,1.019400e+08
81526809,5.304041e+09
79800777,2.802200e+12
...,...
81573964,9.272836e+07
81573965,8.432836e+07
81573966,9.721941e+07
81573967,6.767836e+07


In [142]:
ntx = dict(nx.get_node_attributes(nx_graph,'pr'))
data4 = pd.DataFrame(ntx,index=[0]).T
data4.columns = ['pr']
data4

,pr
81567039,0.000031
81567155,0.000045
81567156,0.000045
81526809,0.000031
79800777,0.033700
...,...
81573964,0.000056
81573965,0.000056
81573966,0.000066
81573967,0.000061


In [143]:
frames = [data,data2,data3,data4]

In [144]:
result = pd.concat(frames)

In [145]:
a = data.join(data3,how='outer')

In [147]:
a

,degree,value
81567039,2,1.139281e+12
81567155,3,2.278358e+12
81567156,1,1.019400e+08
81526809,2,5.304041e+09
79800777,1138,2.802200e+12
...,...,...
81573964,3,9.272836e+07
81573965,3,8.432836e+07
81573966,4,9.721941e+07
81573967,3,6.767836e+07


In [149]:
b= a.join(data2, how='outer')

In [151]:
c = b.join(data4,how='outer')

In [152]:
c

,degree,value,ntx,pr
81567039,2,1.139281e+12,2,0.000031
81567155,3,2.278358e+12,3,0.000045
81567156,1,1.019400e+08,1,0.000045
81526809,2,5.304041e+09,2,0.000031
79800777,1138,2.802200e+12,1654,0.033700
...,...,...,...,...
81573964,3,9.272836e+07,3,0.000056
81573965,3,8.432836e+07,3,0.000056
81573966,4,9.721941e+07,4,0.000066
81573967,3,6.767836e+07,3,0.000061


In [153]:
Gs = StellarGraph.from_networkx(
    nx_graph,node_features=c
)

In [154]:
print(Gs.info())

StellarDiGraph: Directed multigraph
 Nodes: 13093, Edges: 24867

 Node types:
  default: [13093]
    Features: float32 vector, length 4
    Edge types: default-default->default

 Edge types:
    default-default->default: [24867]
        Weights: range=[1, 682], mean=1.44497, std=9.47669
        Features: none


In [155]:
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(Gs)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=False
)

** Sampled 2486 positive and 2486 negative edges. **


In [156]:
# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=False
)

** Sampled 2238 positive and 2238 negative edges. **


In [157]:
batch_size = 20
epochs = 20
num_samples = [20, 10]

In [158]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [159]:
test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [160]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [161]:
# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

/Users/liminghao/miniconda3/envs/stellar/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [162]:
prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [163]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

In [164]:
init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

249/249 [==============================] - 3s 10ms/step - loss: 2.6305 - acc: 0.4783

Train Set Metrics of the initial (untrained) model:
	loss: 2.5641
	acc: 0.4888

Test Set Metrics of the initial (untrained) model:
	loss: 2.6305
	acc: 0.4783


In [165]:
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)

Epoch 1/20
224/224 - 5s - loss: 1.3116 - acc: 0.4696 - val_loss: 1.5247 - val_acc: 0.4743 - 5s/epoch - 22ms/step
Epoch 2/20
224/224 - 4s - loss: 1.0783 - acc: 0.4810 - val_loss: 1.5122 - val_acc: 0.4887 - 4s/epoch - 19ms/step
Epoch 3/20
224/224 - 4s - loss: 1.0729 - acc: 0.4754 - val_loss: 1.5386 - val_acc: 0.4966 - 4s/epoch - 19ms/step
Epoch 4/20
224/224 - 4s - loss: 1.0215 - acc: 0.4864 - val_loss: 1.5303 - val_acc: 0.4956 - 4s/epoch - 19ms/step
Epoch 5/20
224/224 - 4s - loss: 1.0474 - acc: 0.4855 - val_loss: 1.5457 - val_acc: 0.4916 - 4s/epoch - 19ms/step
Epoch 6/20
224/224 - 4s - loss: 1.0037 - acc: 0.4828 - val_loss: 1.5138 - val_acc: 0.4920 - 4s/epoch - 19ms/step
Epoch 7/20
224/224 - 4s - loss: 1.0276 - acc: 0.4832 - val_loss: 1.5131 - val_acc: 0.4899 - 4s/epoch - 19ms/step
Epoch 8/20
224/224 - 4s - loss: 1.0123 - acc: 0.4893 - val_loss: 1.5256 - val_acc: 0.4962 - 4s/epoch - 20ms/step
Epoch 9/20
224/224 - 4s - loss: 1.0495 - acc: 0.4864 - val_loss: 1.5345 - val_acc: 0.4950 - 4s/e